In [ ]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.3.0+cu121
0.18.0+cu121


In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


Bisogna dapprima installare la libreria di roboflow, sito da cui è stato preso il dataset "tennis-ball-detection",
e la libreria ultralytics, la quale consente di utilizzare yolo

In [5]:
!pip install -q roboflow
!pip install -q ultralytics


In [7]:
import ultralytics
print(ultralytics.__version__)
print(ultralytics.__file__)


8.2.25
c:\Users\turch\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\__init__.py


# Get Dataset

In [3]:
#Codice generate da roboflow per importare il dataset di tennis-ball-detection
from roboflow import Roboflow
rf = Roboflow(api_key="sR7Pv1xXBovkKUMHtq3j")
project = rf.workspace("viren-dhanwani").project("tennis-ball-detection")
version = project.version(6)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.25, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to tennis-ball-detection-6 in yolov8:: 100%|██████████| 1168/1168 [00:00<00:00, 1944.41it/s]


In [4]:
import shutil

shutil.move("tennis-ball-detection-6/train",
"tennis-ball-detection-6/tennis-ball-detection-6/train",
)
shutil.move("tennis-ball-detection-6/test",
"tennis-ball-detection-6/tennis-ball-detection-6/test",
)
shutil.move("tennis-ball-detection-6/valid",
"tennis-ball-detection-6/tennis-ball-detection-6/valid",
)

'tennis-ball-detection-6/tennis-ball-detection-6/valid'

In [9]:
print(dataset.location)

/content/tennis-ball-detection-6


Poichè yolo non riconosce bene le palline di tennis, allenerò una rete yolo con un dataset specifico in cui sono presenti dei frame in cui è stata messa in evidenza la posizione della pallina.

In [5]:
from ultralytics import YOLO

# Inizializzare il modello YOLOv8s
model = YOLO("../models/yolov8x.pt")

# Addestrare il modello
model.train(data=f"{dataset.location}/data.yaml", epochs=100, imgsz=640)


Ultralytics YOLOv8.2.25  Python-3.12.0 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce MX230, 2048MiB)
engine\trainer: task=detect, mode=train, model=../models/yolov8x.pt, data=d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\training\tennis-ball-detection-6/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning D:\Linux\Università\Anno III\Lab_IA\Tesi\Tennis-Analyzer\training\tennis-ball-detection-6\tennis-ball-detection-6\train\labels... 428 images, 2 backgrounds, 0 corrupt: 100%|██████████| 428/428 [00:00<00:00, 1111.94it/s]


train: New cache created: D:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\training\tennis-ball-detection-6\tennis-ball-detection-6\train\labels.cache


val: Scanning D:\Linux\Università\Anno III\Lab_IA\Tesi\Tennis-Analyzer\training\tennis-ball-detection-6\tennis-ball-detection-6\valid\labels... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 646.48it/s]


val: New cache created: D:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\training\tennis-ball-detection-6\tennis-ball-detection-6\valid\labels.cache
Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/27 [00:00<?, ?it/s]

Una volta addestrato il modello, nella cartella content/ è presente una cartella runs/detect/train che contiene i pesi del modello e anche metriche e grafici sull'andamento dell'addestramento.